In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors
from datetime import datetime
import ipfx

import os
import pg8000

from ipfx.dataset.create import create_ephys_data_set

from ipfx.data_set_features import extract_data_set_features
from ipfx.utilities import drop_failed_sweeps

import seaborn as sns
from ipfx.epochs import get_stim_epoch

from ipfx.feature_extractor import (
    SpikeFeatureExtractor, SpikeTrainFeatureExtractor
)
from ipfx.stimulus_protocol_analysis import RampAnalysis


In [ ]:
densities_path = "../data/channel_data_densities.xlsx"
df_densities = pd.read_excel(densities_path, sheet_name = 'peak_current_densities')

In [ ]:
print(f"Number of cells with channel rec: {df_densities.shape[0]}")

In [ ]:
# Counts by species

# Human (sorting with Regex)
pattern_H = r'^H[0-9]\d'
df_pattern_H = df_densities.copy()
df_reg_H = df_pattern_H[df_pattern_H['cell'].str.contains(pattern_H, regex=True)]
df_H= df_reg_H.reset_index(drop=True)

# NHP (sorting with Regex)
pattern_QN = r'^QN[0-9]\d'
df_pattern_QN = df_densities.copy()
df_reg_QN = df_pattern_QN[df_pattern_QN['cell'].str.contains(pattern_QN, regex=True)]
df_QN= df_reg_QN.reset_index(drop=True)

#Mouse
pattern_m = r'^(H[0-9]\d|QN[0-9]\d)' #exclude all human and NHP
df_m = df_densities[~df_densities['cell'].str.contains(pattern_m, regex=True)].reset_index(drop=True)

In [ ]:
df_m.head(3)

In [ ]:
# Counts per species
df_m.shape[0] + df_H.shape[0] + df_QN.shape[0] == df_densities.shape[0]
df_m.shape[0], df_H.shape[0], df_QN.shape[0]

## Merge with ttype datasets per species

In [ ]:
try:
    # Human - Use tree_subclass	and tree_cluster for ttype
    h_ttypes_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/shiny/patch_seq/star/human/human_patchseq_MTG_current/mapping.df.lastmap.csv'
    df_human_ttypes = pd.read_csv(h_ttypes_path) 
    df_human_ttypes_reduced = df_human_ttypes[['cell_name_label', 'tree_subclass', 'tree_cluster', 'tree_class']]

    df_human_ch_tt = pd.merge(left = df_H, left_on = 'cell',
                right = df_human_ttypes_reduced, right_on = 'cell_name_label', how = 'inner')


    # Merge with ephys data (query from LIMS)
    df_hum_ephys = pd.read_csv('../data/results_human_query.csv')
    df_human = pd.merge(left = df_human_ch_tt, left_on = 'cell',
                right = df_hum_ephys, right_on = 'name', how = 'inner')

    # Mouse 
    m_ttypes_path = '//allen/programs/celltypes/workgroups/rnaseqanalysis/shiny/patch_seq/star/mouse/mouse_patchseq_WB_current/mapping.df.lastmap.csv'
    df_mouse_ttypes = pd.read_csv(m_ttypes_path)
    df_mouse_ch_tt = pd.merge(left = df_m, left_on = 'cell',
                        right = df_mouse_ttypes, right_on='cell_name', how='inner')

    # Merge with ephys data (query from LIMS)
    df_mouse_ephys = pd.read_csv('../data/results_mouse_query.csv')
    df_mouse = pd.merge(left = df_mouse_ch_tt, left_on = 'cell',
                right = df_mouse_ephys, right_on = 'name', how = 'inner')
    # New col for class
    df_mouse['class'] = df_mouse['best.class_label'].apply(
        lambda x: 'Glutamatergic' if 'Glut' in x else ('GABAergic' if 'GABA' in x else 'Unknown'))

    # Create excel file from dataframes - for remote use.
    df_mouse.to_excel(f'../data/mouse_tt_ch_ephys.xlsx', index=False)
    df_human.to_excel(f'../data/human_tt_ch_ephys.xlsx', index=False)
    print('Data loaded and saved to local files.')
    
except:
    print('Data directory not available. Loading from local files.')
    df_mouse = pd.read_excel('../data/mouse_tt_ch_ephys.xlsx')
    df_human = pd.read_excel('../data/human_tt_ch_ephys.xlsx')

In [ ]:
df_mouse['best.class_label'].unique()

## Number of cells in channel data densities file

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,4))
ax[0].bar(['Mouse','Human','NHP'],[df_m.shape[0], df_H.shape[0], df_QN.shape[0]], color = 'teal', alpha=0.6)
ax[0].set_ylabel('Number of cells')
ax[0].set_xlabel('Species')
ax[0].set_title('Number of cells with channel recording')

ax[1].bar(['Mouse', 'Human'], [df_mouse.shape[0],df_human.shape[0]], color = 'teal', alpha=0.6)
ax[1].set_ylabel('Number of cells')
ax[1].set_xlabel('Species')
ax[1].set_title('Number of cells with channel rec, ttype and ephys')

Parameters mouse:
['cell', 'tot_-50', 'tot_-35', 'tot_-20', 'tot_-5', 'tot_10', 'tot_25',
       'tot_40', 'tot_55', 'tot_70_x', 'tot_85', 'tot_-70', 'tot_-55',
       'tot_-40', 'tot_-25', 'tot_-10', 'tot_5', 'tot_20', 'tot_35', 'tot_50',
       'fast_-50', 'fast_-35', 'fast_-20', 'fast_-5', 'fast_10', 'fast_25',
       'fast_40', 'fast_55', 'fast_70_x', 'fast_85', 'fast_-70', 'fast_-55',
       'fast_-40', 'fast_-25', 'fast_-10', 'fast_5', 'fast_20', 'fast_35',
       'fast_50', 'slow_-50', 'slow_-35', 'slow_-20', 'slow_-5', 'slow_10',
       'slow_25', 'slow_40', 'slow_55', 'slow_70', 'slow_85', 'sus_-50',
       'sus_-35', 'sus_-20', 'sus_-5', 'sus_10', 'sus_25', 'sus_30', 'sus_40',
       'sus_55', 'sus_70', 'sus_85', 'sus_0', 'sus_15', 'sus_30.1', 'sus_45',
       'sus_60', 'sus_75', 'sus_90', 'sus_105', 'sus_120', 'nonfast_-50',
       'nonfast_-35', 'nonfast_-20', 'nonfast_-5', 'nonfast_10', 'nonfast_25',
       'nonfast_40', 'nonfast_55', 'nonfast_70_x', 'nonfast_85', 'nonfast_-70',
       'nonfast_-55', 'nonfast_-40', 'nonfast_-25', 'nonfast_-10', 'nonfast_5',
       'nonfast_20', 'nonfast_35', 'nonfast_50', 'nonfast_-120',
       'nonfast_-105', 'nonfast_-90', 'nonfast_-75', 'nonfast_-60',
       'nonfast_-45', 'nonfast_-30', 'nonfast_-15', 'nonfast_0', 'tot_70_y',
       'fast_70_y', 'fast_total', 'nonfast_70_y', 'nonfast_total', 'jem-date_patch', 'jem-id_species',
       'jem-id_patched_cell_container', 'jem-id_rig_user', 'jem-roi_super',
       'jem-roi_major', 'jem-roi_minor', 'cell_id',
       'rna_amplification_pass_fail', 'Genes.Detected.CPM', 'avg.cor', 'ttype',
       'supertype', 'subclass', 'ephys_qc', 'failed_bad_rs',
       'failed_electrode_0', 'tau', 'avg_isi',
       'upstroke_downstroke_ratio_short_square', 'threshold_v_ramp',
       'peak_v_short_square', 'upstroke_downstroke_ratio_ramp', 'sag',
       'trough_t_ramp', 'slow_trough_v_ramp', 'vrest', 'has_pause',
       'trough_v_long_square', 'input_resistance_mohm',
       'trough_t_short_square', 'fast_trough_v_ramp', 'trough_t_long_square',
       'threshold_v_long_square', 'slow_trough_v_long_square', 'peak_t_ramp',
       'threshold_t_short_square', 'slow_trough_t_long_square',
       'input_access_resistance_ratio', 'fast_trough_t_long_square', 'ri',
       'threshold_t_long_square', 'threshold_v_short_square',
       'threshold_t_ramp', 'vm_for_sag', 'threshold_i_long_square',
       'threshold_i_short_square', 'slow_trough_t_ramp', 'peak_v_ramp',
       'fast_trough_v_short_square', 'fast_trough_t_short_square',
       'fast_trough_t_ramp', 'initial_access_resistance_mohm',
       'slow_trough_t_short_square', 'threshold_i_ramp',
       'slow_trough_v_short_square', 'peak_t_short_square',
       'trough_v_short_square', 'f_i_curve_slope', 'created_at',
       'peak_t_long_square', 'seal_gohm', 'trough_v_ramp', 'latency',
       'fast_trough_v_long_square', 'upstroke_downstroke_ratio_long_square',
       'Unnamed: 72', 'peak_v_long_square', 'adaptation'],

Parameters human:
['cell', 'tot_-50', 'tot_-35', 'tot_-20', 'tot_-5', 'tot_10', 'tot_25',
       'tot_40', 'tot_55', 'tot_70', 'tot_85', 'tot_-70', 'tot_-55', 'tot_-40',
       'tot_-25', 'tot_-10', 'tot_5', 'tot_20', 'tot_35', 'tot_50', 'fast_-50',
       'fast_-35', 'fast_-20', 'fast_-5', 'fast_10', 'fast_25', 'fast_40',
       'fast_55', 'fast_70', 'fast_85', 'fast_-70', 'fast_-55', 'fast_-40',
       'fast_-25', 'fast_-10', 'fast_5', 'fast_20', 'fast_35', 'fast_50',
       'slow_-50', 'slow_-35', 'slow_-20', 'slow_-5', 'slow_10', 'slow_25',
       'slow_40', 'slow_55', 'slow_70', 'slow_85', 'sus_-50', 'sus_-35',
       'sus_-20', 'sus_-5', 'sus_10', 'sus_25', 'sus_30', 'sus_40', 'sus_55',
       'sus_70', 'sus_85', 'sus_0', 'sus_15', 'sus_30.1', 'sus_45', 'sus_60',
       'sus_75', 'sus_90', 'sus_105', 'sus_120', 'nonfast_-50', 'nonfast_-35',
       'nonfast_-20', 'nonfast_-5', 'nonfast_10', 'nonfast_25', 'nonfast_40',
       'nonfast_55', 'nonfast_70', 'nonfast_85', 'nonfast_-70', 'nonfast_-55',
       'nonfast_-40', 'nonfast_-25', 'nonfast_-10', 'nonfast_5', 'nonfast_20',
       'nonfast_35', 'nonfast_50', 'nonfast_-120', 'nonfast_-105',
       'nonfast_-90', 'nonfast_-75', 'nonfast_-60', 'nonfast_-45',
       'nonfast_-30', 'nonfast_-15', 'nonfast_0', 'seq-name', 'cell_id',
       'patched_cell_container_label', 'cell_name_label', 'batch_vendor_name_label',
       'SEAAD_class_label.assignment',
       'SEAAD_class_label.bootstrapping_probability',
       'SEAAD_class_label.avg_correlation', 'SEAAD_subclass_label.assignment',
       'SEAAD_subclass_label.bootstrapping_probability',
       'SEAAD_subclass_label.avg_correlation',
       'SEAAD_cluster_label.assignment',
       'SEAAD_cluster_label.bootstrapping_probability',
       'SEAAD_cluster_label.avg_correlation', 'SEAAD_score.Corr',
       'SEAAD_cluster_Corr', 'SEAAD_subclass_Corr', 'SEAAD_class_Corr',
       'rna_amplification_pass_fail', 'library_prep_pass_fail',
       'percent_reads_aligned_to_exons', 'percent_reads_aligned_to_mt_exons',
       'percent_reads_aligned_to_introns',
       'percent_reads_aligned_to_intergenic', 'percent_reads_aligned_unique',
       'percent_reads_aligned_to_ecoli',
       'percent_reads_aligned_to_synthetic_constructs',
       'percent_reads_aligned_total', 'complexity_aatt', 'complexity_acgt',
       'complexity_agct', 'complexity_at', 'complexity_catg',
       'complexity_ccgg', 'complexity_cg', 'complexity_gatc', 'complexity_gc',
       'complexity_ta', 'fpkm_published', 'fpkm_dir', 'fpkm_filename',
       'Genes.Detected', 'Exclude.STAR', 'TreeMatch_1', 'TreeProb_1',
       'TreeMatch_2', 'TreeProb_2', 'TreeMatch_3', 'TreeProb_3', 'score.Corr',
       'score.Tree', 'cluster_Corr', 'subclass_Corr', 'class_Corr', 'cluster_Tree', 'subclass_Tree', 'class_Tree', 'cluster',
       'cluster_score', 'res.index', 'quality_score', 'marker_sum_norm',
       'Norm_Marker_Sum.0.4', 'contaminationType', 'contam_sum', 'Tree_call',
       'Tree_first_cor', 'Tree_first_bt', 'Tree_second_bt', 'Tree_first_KL',
       'Tree_second_KL', 'seuratQCpass', 'UMAP_x', 'UMAP_y',
       'class_MapMyCells', 'class_MapMyCells_softmax_probability',
       'subclass_MapMyCells', 'subclass_MapMyCells_softmax_probability',
       'supertype_MapMyCells', 'supertype_MapMyCells_softmax_probability'

In [ ]:
classification_m = 'subclass' # 'ttype','supertype', 'subclass'
classification_h = 'tree_subclass' # 'tree_cluster', 'tree_subclass', 'tree_class'

fig, ax = plt.subplots(1,2,figsize=(10,7), gridspec_kw={'wspace': 0.35},
                      width_ratios= [2,0.8] )
for i, species, classification, df_species in zip([0,1],['Mouse', 'Human'],
                                            [classification_m, classification_h],
                                            [df_mouse, df_human]):
    
    ax[i].barh(df_species[classification].value_counts().index,
               df_species[classification].value_counts(), color = 'teal',
               alpha=0.6)
    ax[i].tick_params(axis='both', labelsize=8)
    ax[i].set_xlabel('Number of cells')
    ax[i].set_ylabel(f'{classification}')
    ax[i].set_title(f'{species} cells with channel recording')

### Samples for the dashboard

#### Scatterplots of columns

In [ ]:
plt.scatter(df_mouse['tot_70'], df_mouse['upstroke_downstroke_ratio_ramp'], color = 'teal', alpha=0.6)

In [ ]:
plt.scatter(df_human['tot_70'], df_human['upstroke_downstroke_ratio_ramp'], color = 'teal', alpha=0.6)